In [1]:
! pip install langchain langchain-community langchain-openai



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! pip install python-dotenv



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
! pip install sentence-transformers faiss-cpu langchain huggingface-hub



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
! pip install langchain langchain-experimental



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
! pip install tabulate
! pip install tabulate matplotlib seaborn numpy



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


# .env 読み込み
print(r"..\environment\.env")
load_dotenv(dotenv_path=r'..\environment/.env',override=True)

# =========================
# 🔐 Metabase 認証・データ取得
# =========================

login_url = "https://metabase.tr-com.net/api/session"
username = os.getenv("MB_USERNAME", "").strip()
password = os.getenv("MB_PASSWORD", "").strip()

if not username or not password:
    raise ValueError("❌ MB_USERNAME または MB_PASSWORD が設定されていません")

print(f"ログイン試行中 → username: '{username}'")

response = requests.post(login_url, json={"username": username, "password": password})
if response.status_code == 200:
    session_id = response.json().get("id")
    print("✅ ログイン成功")
else:
    raise Exception(f"❌ ログイン失敗: {response.status_code}, {response.text}")


..\environment\.env
ログイン試行中 → username: 'y-nitta@torishima.co.jp'
✅ ログイン成功


In [7]:
# API URL
url = "https://metabase.tr-com.net/api/card/2061/query/json"


headers = {
    "Content-Type": "application/json",
    "X-Metabase-Session": session_id
}

# Payload
payload = {
    "parameters": [
        {
            "type": "category",
            "target": ["variable", ["template-tag", "sensorid"]],
            "value": "XP1VC000D1C"
        }
    ]
}

# Make the request
response = requests.post(url, headers=headers, json=payload)


In [8]:

response = requests.post(url, headers=headers)
if response.status_code == 200:
    try:
        data = response.json()
        df = pd.DataFrame(data)
        if df.empty:
            raise ValueError("❌ 取得したデータが空です。Metabaseを確認してください。")
        print("✅ データ取得成功")
        print(df.head())

    except Exception as e:
        raise Exception(f"❌ データ取得・DataFrame変換失敗: {e}")
else:
    raise Exception(f"❌ データ取得失敗: {response.status_code}, {response.text}")


✅ データ取得成功
      sensorid                     UserId            TimeStamp  \
0  XP1VC000D1C  096210@pnet.energia.co.jp  2023-07-24T05:59:02   
1  XP1VC000D1C  096210@pnet.energia.co.jp  2023-07-24T05:59:34   
2  XP1VC000D1C  094000@pnet.energia.co.jp  2023-07-31T06:40:07   
3  XP1VC000D1C  094000@pnet.energia.co.jp  2023-08-07T05:32:47   
4  XP1VC000D1C  270103@pnet.energia.co.jp  2023-08-14T06:49:07   

            recordDate     VRMSX    VRMSXAcc  Temperature  
0  2023-07-24T05:59:02  0.027953  194.881890        42.56  
1  2023-07-24T05:59:33  0.030709  320.078740        44.54  
2  2023-07-31T06:40:07  0.027559    6.299213        35.53  
3  2023-08-07T05:32:47  0.027559    5.511811        29.98  
4  2023-08-14T06:49:07  0.027559    5.511811        30.87  


In [ ]:



# =========================
# 🤖 Pandas Agent 設定
# =========================

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2,
    max_tokens=500,
)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True  
)

# =========================
# 💬 質問関数と会話ループ
# =========================

def ask_question(question: str):
    try:
        response = agent.run(question)
        print(f"👤 {question}")
        print(f"🤖 {response}\n")
    except Exception as e:
        print(f"❌ エラー: {e}")

def talk():
    print("🔁 DataFrame QAエージェント開始（'exit' または '終了' で終了）")
    while True:
        try:
            user_input = input("👤 ")
            if user_input.lower() in ["exit", "終了", "quit"]:
                print("🛑 終了します")
                break
            response = agent.run(user_input)
            print(f"🤖 {response}\n")
        except KeyboardInterrupt:
            print("\n🛑 中断されました。会話を終了します")
            break

# =========================
# ✅ 使用例
# =========================

# ask_question("3列目の最頻値は？")
# talk()
ask_question("何ができる？")

C:\Users\H6445\AppData\Local\Temp\ipykernel_10132\3402648702.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
C:\Users\H6445\AppData\Local\Temp\ipykernel_10132\3402648702.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)




> Entering new AgentExecutor chain...
Thought: この質問は、データフレーム `df` に対してどのような操作や分析が可能かを尋ねています。データフレームの内容を確認し、どのような情報を抽出できるかを考えます。具体的には、列の情報やデータ型、基本的な統計量などを確認することができます。
Action: python_repl_ast
Action Input: df.info()<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sensorid     51 non-null     object 
 1   UserId       51 non-null     object 
 2   TimeStamp    51 non-null     object 
 3   recordDate   51 non-null     object 
 4   VRMSX        51 non-null     float64
 5   VRMSXAcc     51 non-null     float64
 6   Temperature  51 non-null     float64
dtypes: float64(3), object(4)
memory usage: 2.9+ KB
データフレーム `df` には、51行と7列のデータが含まれています。各列のデータ型は以下の通りです：

- `sensorid`: オブジェクト型（文字列）
- `UserId`: オブジェクト型（文字列）
- `TimeStamp`: オブジェクト型（文字列）
- `recordDate`: オブジェクト型（文字列）
- `VRMSX`: 浮動小数点型
- `VRMSXAcc`: 浮動小数点型
- `Temperature`: 浮動小数点型

このデータフレームに対してできることは以下の通りです：

In [10]:
ask_question("特徴的なデータを探して（理由も）")




> Entering new AgentExecutor chain...
Thought: 特徴的なデータを探すためには、データフレームの各列の統計情報や分布を確認する必要があります。特に、温度や電圧の値が異常に高いまたは低い場合、または特定のユーザーやセンサーに関連するパターンがあるかどうかを調べることが重要です。まずは、各列の基本的な統計情報を確認します。

Action: python_repl_ast
Action Input: df.describe()           VRMSX    VRMSXAcc  Temperature
count  51.000000   51.000000    51.000000
mean    0.025992   44.163965    29.262941
std     0.003089   62.238340     5.575400
min     0.019291    2.755906    18.150000
25%     0.022835    4.921260    25.125000
50%     0.026772    6.299213    28.470000
75%     0.027559   75.590551    32.065000
max     0.031890  320.078740    44.540000データの基本的な統計情報を確認した結果、以下のことがわかりました：

- **VRMSX**: 最小値は0.019291、最大値は0.031890で、平均は0.025992です。標準偏差は0.003089で、比較的狭い範囲に収束しています。
- **VRMSXAcc**: 最小値は2.755906、最大値は320.078740で、平均は44.163965です。標準偏差は62.238340と非常に大きく、データに異常値が含まれている可能性があります。
- **Temperature**: 最小値は18.150000、最大値は44.540000で、平均は29.262941です。標準偏差は5.575400で、温度の変動は比較的少ないですが、最大値が高めです。

特に注目すべきは、**VRMSXAcc**の最大値320.078740です。この値は他のデータポイントに比べ